In [ ]:
## 0. load packages
import os, sys
import numpy as np
import pandas as pd
import xarray as xr
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib.dates as dates
import matplotlib.ticker as ticker
import matplotlib.colors as colors
import seaborn as sns
from scipy.stats import norm
from datetime import datetime

os.chdir('/h/u145/liuxinrui/CROP/')
sys.path.insert(0, '/h/u145/liuxinrui/CROP/')

impact_models =  ['CYGMA1p74', 'EPIC-IIASA', 'ISAM', 'LDNDC', 'LPJmL', 'PEPIC', 'PROMET', 'SIMPLACE-LINTUL5']
spcs = ['mai', 'ri1', 'ri2', 'soy', 'swh', 'wwh']

In [ ]:
## 1. Load parameters

current_datetime = datetime.now()
print(f'\033[1;31;46mCurrent date and time: {current_datetime}\033[0m')

from core_fct.fct_loadP_CROP import load_ISIMIP3b_param
Par = load_ISIMIP3b_param()
Par['N_bnf'] = Par['N_bnf'].sel(mod_bnf_soy='Ma_2022', drop=True)
Par = Par.drop_dims(['mod_bnf_soy'])

## change the orginal Parameter format
from core_fct.fct_genMC import extrat_crop_par, restore_crop_par, generate_config
print(Par['YD_0'].sel(spc_crop='ri1', reg_land=287, irr='noirr').values)
Par1 = extrat_crop_par(Par)
Par0 = restore_crop_par(Par1)
print(Par0['YD_0'].sel(spc_crop='ri1', reg_land=287, irr='noirr').values)
Par2 = generate_config(Par1, nMC=1000, kde_to_mod=False, mod_to_unc=False, mod_noise=0.1, seed=1997)
Par = restore_crop_par(Par2)
print(Par['YD_0'].sel(spc_crop='ri1', reg_land=287, irr='noirr').values)

Current date and time: 2025-04-23 11:22:00.912752
loading primary parameters
>>> Running split_region <<<
>>> Running split_region <<<
>>> Running split_region <<<
>>> Running split_region <<<
>>> Running split_region <<<
>>> Running split_region <<<
>>> Running split_region <<<
>>> Running split_region <<<
>>> Running split_region <<<
>>> Running split_region <<<
>>> Running split_region <<<
>>> Running split_region <<<
>>> Running split_region <<<
>>> Running split_region <<<
>>> Running split_region <<<
>>> Running split_region <<<
[6.84068774 2.92453226 3.526445   3.11523153 2.91071177 4.68409691
 2.55204919        nan]
extracting crop emulator parameters
restoring crop emulator parameters
[6.84068774 2.92453226 3.526445   3.11523153 2.91071177 4.68409691
 2.55204919        nan]
generating MC configurations
restoring crop emulator parameters
[2.59042728 3.31723606 6.67813325 3.50136779 4.763478   6.94436072
 3.22165452 2.88785414 3.67975885 3.12715568 6.84578856 7.07935784
 4.61388

In [ ]:
## 2. load drivers

from core_fct.fct_loadD_CROP import load_Nfertl_scen, load_Ndep_scen
soc = '2015soc'
For_Nfertl = load_Nfertl_scen(datasets=['ISIMIP3b']).sel(scen_N_fertl='ISIMIP3b-'+soc, drop=True).isel(year=0, drop=True)
print(For_Nfertl)

For_Ndep = load_Ndep_scen(datasets=['ISIMIP3b']).sel(scen_N_dep='ISIMIP3b-'+soc, drop=True)
print(For_Ndep)

For_clim_0 = xr.load_dataset('./input_data/observations/crop/land-climate_ISIMIP3b.nc').sel(scen='historical', drop=True).dropna('year', how='all')
For_clim_1 = xr.load_dataset('./input_data/observations/crop/land-climate_ISIMIP3b.nc').sel(scen='ssp585', drop=True).dropna('year', how='all')
For_co2_0 = xr.load_dataset('./input_data/observations/crop/concentrations_ISIMIP3b.nc').sel(scen='historical', drop=True).dropna('year', how='all')
For_co2_1 = xr.load_dataset('./input_data/observations/crop/concentrations_ISIMIP3b.nc').sel(scen='ssp585', drop=True).dropna('year', how='all')
For_clim = xr.concat([For_clim_0, For_clim_1], dim='year')
For_co2 = xr.concat([For_co2_0, For_co2_1], dim='year')

## set run options
ind0, ind1 = 1850, 2100
For = xr.merge([For_clim, For_co2, For_Nfertl, For_Ndep]).sel(year=slice(ind0, ind1))

## format drivers
## Initial CO2 here is 399.95 ppm
For['D_CO2'] = For.CO2 - Par['CO2_0']
For['D_Tl'] = For.Tl - For.Tl.sel(year=slice(1851, 1880)).mean('year')
For['D_Pl'] = For.Pl - For.Pl.sel(year=slice(1851, 1880)).mean('year')
For = For.drop_vars(['CO2', 'Tl', 'Pl'])
For = For.fillna(0).sel(year=slice(1850)).combine_first(For)

In [ ]:
## 3. import offline crop emulator and run emulations

from core_fct.mod_process_CROP import CROP
CROP = CROP()

print(CROP.proc_levels())
print(CROP.var_in)
print(CROP.var_mid)
print(CROP.var_out)
print(CROP._processes.keys())
print(CROP.var_prog)
print(CROP.var_node)
print(CROP.var_diag)

Out = CROP(None, Par, For, var_keep=['D_Tgs', 'D_Pgs', 'NI', 'RC', 'RT', 'RP', 'RN', 'YD', 'YD_0'])
for var in Out.data_vars:
    Out[var].to_netcdf(f'./results/new/{var}_noise.nc')

print(Out['RC'].max(), Out['RC'].min())
print(Out['RT'].max(), Out['RT'].min())
print(Out['RP'].max(), Out['RP'].min())
print(Out['RN'].max(), Out['RN'].min())
print(Out['YD'].max(), Out['YD'].min())